In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import replicate
import json

In [4]:
from pydub import AudioSegment
import io

This is the first 25ish minutes of the webinar

In [5]:
# audio = AudioSegment.from_file("./data/youtube/langchain-openai-webinar-clip.mkv")

In [6]:
def getFileNames(splitNumber):
    fileId = "langchain-webinar first split {}".format(splitNumber)
    return "./data/youtube/{}.mp3".format(fileId), "./data/youtube/{} - Replicate x Whisper API Response.json".format(fileId)

In [7]:
# chunk_duration = 10
# chunk_duration_ms = chunk_duration * 60 * 1000
# for split_number, i in enumerate(range(0, len(audio), chunk_duration_ms)):
# 	# Audio chunk
# 	chunk = audio[i : i + chunk_duration_ms]

# 	audioFilePath, responseFilePath = getFileNames(split_number)

# 	chunk.export(audioFilePath, format="mp3")

# 	output = replicate.run(
# 		"openai/whisper:91ee9c0c3df30478510ff8c8a3a545add1ad0259ad3a9f78fba57fbc05ee64f7",
# 		input={"audio": open(audioFilePath, "rb")}
# 	)
# 	with open(responseFilePath, 'w') as f:
# 		json.dump(output, f)

In [8]:
segments = []
for i in range(3):
	_, responseFilePath = getFileNames(i)
	output = json.load(open(responseFilePath, 'r'))
	segments += output['segments']	

In [10]:
transcript = "\n".join([segment['text'] for segment in segments])
transcript = transcript.strip()

In [11]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

In [12]:
textSplitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [13]:
transcriptChunks = textSplitter.create_documents([transcript])

Add consecutive segments of the video as `metadata` to each chunk of the transcript

In [32]:
SEGMENT_OVERLAP_SECONDS = 3
chunksWithSegments = []

for chunk in transcriptChunks:
    chunkText = chunk.page_content

    chunkSegments = []
    for segment in segments:
        if segment['text'].strip() in chunkText:
            chunkSegments.append(segment)

    # Store consecutive segments as one segment
    consecutiveSegments = [{
        'start': chunkSegments[0]['start'],
        'end': chunkSegments[0]['end']
    }]

    for segment in chunkSegments[1:]:
        lastSegmentEnd = consecutiveSegments[-1]['end']

        # range doesn't work for floats
        if segment['start'] >= lastSegmentEnd and segment['start'] <= lastSegmentEnd + SEGMENT_OVERLAP_SECONDS:
            consecutiveSegments[-1]['end'] = segment['end']
        else:
            consecutiveSegments.append({
                'start': segment['start'],
                'end': segment['end']
            })
    
    chunksWithSegments.append(Document(
        page_content=chunkText,
        metadata={
        'blockId': "k3u46gu4bg",
        'segments': json.dumps(consecutiveSegments)
        })
    )


In [33]:
# db = Chroma.from_documents(documents=chunksWithSegments, embedding=OpenAIEmbeddings(), persist_directory="./langchain_webinar_db")
# db.persist()
# db = None

In [34]:
db = Chroma(persist_directory="./langchain_webinar_db", embedding_function=OpenAIEmbeddings())

In [38]:
db.similarity_search("sql injection")

[Document(page_content="interact with its own inputs. And I'll show an example of doing citations.\n These are kind of like pretty atrocious abuse of the function call API, but I\n think these are some cool examples worth showing off. The first example I'll show\n is one around maybe preventing SQL injection. Right now when we use SQL\n agents, they kind of just output the SQL with like no escaped values. But we know\n that to call SQL safely, we want to have template strings and we want to have\n query parameters. So if we model this, so you know, a SQL template has a literal\n or an identifier. There are parameters that have keys and values with a type.\n And the SQL query is actually a template and its parameters. And just for\n safety, I add a is dangerous flag that determines whether or not I think or the\n model thinks there's any kind of SQL injection. So this is all kind of just\n modeling the data. What PyData can do is it generates the function schema", metadata={'blockId': '

In [35]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI


qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-3.5-turbo-0613"),
    chain_type="stuff",
    retriever=db.as_retriever(),
    return_source_documents=True
)

In [41]:
res = qa("When did jason introduce himself?")
print(res['result'])

Jason introduced himself in the conversation when he said, "So I'm Jason. I spent the past, like maybe eight, 10 years doing things in machine learning around computer vision and recommendation systems."


In [42]:
for source in res['source_documents']:
	print(source.page_content)
	print(source.metadata['segments'])
	print()

what I'm playing with right now.
 Awesome.
 Jason, do you want to do a quick intro?
 Cool, yeah.
 So I'm Jason.
 I spent the past, like maybe eight, 10 years
 doing things in machine learning
 around computer vision and recommendation systems.
 And I almost neglected natural language processing
 because I thought it was kind of like a boring subject.
 And coming back to it, I kind of regret doing that.
 And now I think language models
 are sort of like the coolest thing out there.
 So I remember playing with GBD2,
 thinking it was cool,
 then going back to my actual work.
 And now GBD4, I'm kind of like asking nicely,
 saying please, and doing everything I can
 to get the answer out.
 So it's been a fun ride.
 All right.
 Ati, do you want to go?
 Sure.
 Hi everyone.
 My name is Ati.
 I'm an engineer at OpenAI.
 I help build chat completions and function calling.
 And it's crazy to see more than a thousand people here.
 So I'm excited to share more today.
 And David.
 Hi everyone.
[{"st